In [9]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meirnizri/cellphones-recommendations")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cellphones-recommendations


In [5]:
# Load
df = pd.read_csv("/kaggle/input/cellphones-recommendations/cellphones ratings.csv")

# Encode IDs
user2idx = {id: i for i, id in enumerate(df['user_id'].unique())}
item2idx = {id: i for i, id in enumerate(df['cellphone_id'].unique())}

df['user'] = df['user_id'].map(user2idx)
df['item'] = df['cellphone_id'].map(item2idx)

# Train/test split
train, test = train_test_split(df[['user', 'item', 'rating']], test_size=0.2)


In [6]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim=50):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, latent_dim)
        self.item_emb = nn.Embedding(num_items, latent_dim)

    def forward(self, u, i):
        return (self.user_emb(u) * self.item_emb(i)).sum(1)


In [7]:
model = MF(len(user2idx), len(item2idx))
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

u = torch.tensor(train['user'].values)
i = torch.tensor(train['item'].values)
r = torch.tensor(train['rating'].values, dtype=torch.float)

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    pred = model(u, i)
    loss = loss_fn(pred, r)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: loss = {loss.item():.4f}")


Epoch 1: loss = 93.5308
Epoch 2: loss = 89.2844
Epoch 3: loss = 85.1821
Epoch 4: loss = 81.2225
Epoch 5: loss = 77.4035
Epoch 6: loss = 73.7229
Epoch 7: loss = 70.1778
Epoch 8: loss = 66.7652
Epoch 9: loss = 63.4822
Epoch 10: loss = 60.3254


In [8]:
def recommend(user_id, top_k=5):
    model.eval()
    u_idx = torch.tensor([user2idx[user_id]] * len(item2idx))
    i_idx = torch.tensor(list(item2idx.values()))
    scores = model(u_idx, i_idx).detach().numpy()
    top_items = sorted(zip(item2idx.keys(), scores), key=lambda x: -x[1])[:top_k]
    return [item for item, _ in top_items]

# Example
print("Top picks:", recommend(user_id=258))


Top picks: [np.int64(21), np.int64(17), np.int64(23), np.int64(20), np.int64(24)]
